# Evaluating Performance - House Prices Model

## By Jean-Philippe Pitteloud

### Requirements

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import statsmodels.api as sm
from scipy.stats import jarque_bera
from scipy.stats import normaltest
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### Data Gathering

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_df = pd.read_sql_query('select * from houseprices',con = engine)

engine.dispose()


house_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


### Modeling and Evaluation

#### _Model #1:_

In our first and simpler model, a Linear Regression Model is used using OLS method. The target variable is 'saleprice'. The independent variables included are 'overallqual', 'grlivarea', 'garagearea', 'firstflrsf', and the categorical variables 'exterqual', 'kitchenqual', and 'mszoning'. Upon creating dummy variables for all three categorical variables, the model was estimated and the results and statistics presented below

In [3]:
house_df = pd.concat([house_df,pd.get_dummies(house_df['exterqual'], prefix = 'exterqual_dummy', drop_first=True)], axis = 1)

house_df = pd.concat([house_df,pd.get_dummies(house_df['kitchenqual'], prefix = 'kitchenqual_dummy', drop_first=True)], axis = 1)

house_df = pd.concat([house_df,pd.get_dummies(house_df['mszoning'], prefix = 'mszoning_dummy', drop_first=True)], axis = 1)

In [4]:
X_1 = house_df[['overallqual', 'grlivarea', 'garagearea', 'firstflrsf', 'exterqual_dummy_Fa', 'exterqual_dummy_Gd', 'exterqual_dummy_TA', 'kitchenqual_dummy_Fa', 'kitchenqual_dummy_Gd', 'kitchenqual_dummy_TA', 'mszoning_dummy_FV', 'mszoning_dummy_RH', 'mszoning_dummy_RL', 'mszoning_dummy_RM']]

Y_1 = house_df['saleprice']

X_1 = sm.add_constant(X_1)

results_1 = sm.OLS(Y_1, X_1).fit()

results_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     398.0
Date:                Mon, 14 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:19:55   Log-Likelihood:                -17390.
No. Observations:                1460   AIC:                         3.481e+04
Df Residuals:                    1445   BIC:                         3.489e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  1.34e+04   1.58e+04      0.848      0.397   -1.76e+04    4.44e+04
overallqual           1.784e+04   1191.975     14.970      0.000    1.55e+04    2.02e+04
grlivarea               42.5695      2.472     17.220      0.000      37.720      47.419
garagearea              46.8522      5.831      8.035      0.000      35.414      58.290
firstflrsf              18.9033      3.326      5.684      0.000      12.379      25.427
exterqual_dummy_Fa   -4.351e+04    1.3e+04     -3.336      0.001   -6.91e+04   -1.79e+04
exterqual_dummy_Gd   -3.636e+04   6388.306     -5.692      0.000   -4.89e+04   -2.38e+04
exterqual_dummy_TA   -5.005e+04   7055.326     -7.093      0.000   -6.39e+04   -3.62e+04
kitchenqual_dummy_Fa -4.894e+04   8298.691     -5.898      0.000   -6.52e+04   -3.27e+04
kitchenqual_dummy_Gd -3.703e+04   4777.888     -7.751      0.000   -4.64e+04   -2.77e+04
kitchenqual_dummy_TA -4.692e+04   5352.308     -8.767      0.000   -5.74e+04   -3.64e+04
mszoning_dummy_FV     3.018e+04   1.29e+04      2.336      0.020    4842.072    5.55e+04
mszoning_dummy_RH     2.006e+04   1.48e+04      1.356      0.175   -8964.750    4.91e+04
mszoning_dummy_RL     3.725e+04    1.2e+04      3.101      0.002    1.37e+04    6.08e+04
mszoning_dummy_RM     1.691e+04   1.21e+04      1.393      0.164   -6895.736    4.07e+04
==============================================================================
Omnibus:                      532.994   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            61387.504
Skew:                          -0.664   Prob(JB):                         0.00
Kurtosis:                      34.739   Cond. No.                     5.94e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.94e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

As it can be seen in the table above, the first model is able to explain 79% of the variance in the target. Most of the independent variables included were judged as statistically significant, with the exception of the introduced constant and two dummy variables for the 'mszoning' variable. In the case of the two dummy variables, the p-values were close to the acceptable 0.1 values while the p-value associated with the constant was 0.397

In terms of the F-statistic, a value of 398 was obtained, confirming the ability of our model to explain more variance in the target than an "empty model". A F-statistic close to zero suggest the difference in explanatory power between our model and the "empty" model is statistically significant.

In terms of the metrics Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC), the model above estimated values for both AIC and BIC of 34,810. These values take into account the Sum of the Squares Errors (SSE) along with the sample size and the number of parameters. These two metrics are very useful in comparing the performance of models, so in the next two sections, modified models will be estimated and the statistics compared to the ones discussed in this section

#### _Model #2:_

In the second model, the target variable was the same ('saleprice') while the two dummy variables 'mszoning_RH' and 'mszoning_RM' (previously found to be non-statistically significant) were removed from the model. The model was fitted and evaluated below

In [5]:
X_2 = house_df[['overallqual', 'grlivarea', 'garagearea', 'firstflrsf', 'exterqual_dummy_Fa', 'exterqual_dummy_Gd', 'exterqual_dummy_TA', 'kitchenqual_dummy_Fa', 'kitchenqual_dummy_Gd', 'kitchenqual_dummy_TA', 'mszoning_dummy_FV', 'mszoning_dummy_RL']]

Y_2 = house_df['saleprice']

X_2 = sm.add_constant(X_2)

results_2 = sm.OLS(Y_2, X_2).fit()

results_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     464.1
Date:                Mon, 14 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:19:55   Log-Likelihood:                -17392.
No. Observations:                1460   AIC:                         3.481e+04
Df Residuals:                    1447   BIC:                         3.488e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 2.898e+04   1.13e+04      2.575      0.010    6908.277    5.11e+04
overallqual             1.8e+04   1186.449     15.175      0.000    1.57e+04    2.03e+04
grlivarea               42.5568      2.470     17.231      0.000      37.712      47.402
garagearea              46.1209      5.810      7.939      0.000      34.725      57.517
firstflrsf              18.9744      3.325      5.707      0.000      12.452      25.496
exterqual_dummy_Fa     -4.6e+04   1.28e+04     -3.583      0.000   -7.12e+04   -2.08e+04
exterqual_dummy_Gd   -3.625e+04   6388.159     -5.675      0.000   -4.88e+04   -2.37e+04
exterqual_dummy_TA    -4.98e+04   7053.676     -7.060      0.000   -6.36e+04    -3.6e+04
kitchenqual_dummy_Fa -4.884e+04   8297.715     -5.886      0.000   -6.51e+04   -3.26e+04
kitchenqual_dummy_Gd -3.693e+04   4777.508     -7.730      0.000   -4.63e+04   -2.76e+04
kitchenqual_dummy_TA -4.685e+04   5351.733     -8.755      0.000   -5.74e+04   -3.64e+04
mszoning_dummy_FV     1.362e+04   5403.251      2.520      0.012    3016.102    2.42e+04
mszoning_dummy_RL     2.071e+04   2728.337      7.591      0.000    1.54e+04    2.61e+04
==============================================================================
Omnibus:                      532.909   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            60863.110
Skew:                          -0.667   Prob(JB):                         0.00
Kurtosis:                      34.602   Cond. No.                     3.50e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.5e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

As it can be seen in the summary table above, the adjusted R-squared value obtained was 0.794 suggesting that this new model explains the same proportion of variance in the target variable as Model #1. In this particular model, all features were considered statistically significant as judged by their associated p-values. In terms of the F-statistic obtained from the F-test, this new model, in which two features from the original model were removed, is associated to a F-statistic value of 464.1. The comparison of this value to the value obtained from Model #1, suggest that model #2 more efficiently explains the variance than Model #1, although the adjusted R-squared values are identical

Regarding the AIC and BIC metrics, the evaluation of the new model gave values that are virtually identical to the ones obtained for Model #1

#### _Model #3:_

In our third model, the target variable was not changed while two additional predictor variables were included, 'lotarea' and 'fireplaces'. The new model was fitted and evaluated and the summary of the results is presented below

In [6]:
X_3 = house_df[['overallqual', 'grlivarea', 'garagearea', 'firstflrsf', 'lotarea', 'fireplaces', 'exterqual_dummy_Fa', 'exterqual_dummy_Gd', 'exterqual_dummy_TA', 'kitchenqual_dummy_Fa', 'kitchenqual_dummy_Gd', 'kitchenqual_dummy_TA', 'mszoning_dummy_FV', 'mszoning_dummy_RL']]

Y_3 = house_df['saleprice']

X_3 = sm.add_constant(X_3)

results_3 = sm.OLS(Y_3, X_3).fit()

results_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                     416.4
Date:                Mon, 14 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:19:55   Log-Likelihood:                -17364.
No. Observations:                1460   AIC:                         3.476e+04
Df Residuals:                    1445   BIC:                         3.484e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  3.87e+04   1.12e+04      3.452      0.001    1.67e+04    6.07e+04
overallqual           1.756e+04   1184.699     14.821      0.000    1.52e+04    1.99e+04
grlivarea               38.0041      2.512     15.130      0.000      33.077      42.931
garagearea              45.2454      5.720      7.910      0.000      34.026      56.465
firstflrsf              14.3048      3.325      4.302      0.000       7.782      20.828
lotarea                  0.5606      0.101      5.530      0.000       0.362       0.760
fireplaces            7118.3812   1723.729      4.130      0.000    3737.103    1.05e+04
exterqual_dummy_Fa   -4.883e+04   1.26e+04     -3.870      0.000   -7.36e+04   -2.41e+04
exterqual_dummy_Gd   -3.699e+04   6276.345     -5.894      0.000   -4.93e+04   -2.47e+04
exterqual_dummy_TA   -5.212e+04   6936.956     -7.514      0.000   -6.57e+04   -3.85e+04
kitchenqual_dummy_Fa -4.909e+04   8149.668     -6.024      0.000   -6.51e+04   -3.31e+04
kitchenqual_dummy_Gd   -3.7e+04   4693.625     -7.883      0.000   -4.62e+04   -2.78e+04
kitchenqual_dummy_TA -4.745e+04   5257.743     -9.024      0.000   -5.78e+04   -3.71e+04
mszoning_dummy_FV     1.392e+04   5310.843      2.621      0.009    3499.407    2.43e+04
mszoning_dummy_RL     1.757e+04   2712.753      6.477      0.000    1.23e+04    2.29e+04
==============================================================================
Omnibus:                      584.575   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            78311.813
Skew:                          -0.804   Prob(JB):                         0.00
Kurtosis:                      38.843   Cond. No.                     2.49e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.49e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

From the table above, we can conclude that this third most succesfully explains the variance in the target with an adjusted R-squared of 0.799. This R-square value represents an improvement in the explanatory power compared to both Model #1 and Model #2 (79%). Once again all included features were estimated as statistically significant. 

Concerning the F-statistic value, this new model is associated to a value of 416.4 while Model #2 was associated to a larger value of 464.1. Model #2 can be considered to be nested on Model #3 (additional 2 features). Judging by the F-statistic values, Model #2 may be seen as more efficient explaining the variance in the target variable than Model #3, however, the slight gain in R-squared value may balance this out. Remember the F-statistics considers the performance of the model in comparison to an "empty" model

In terms of the AIC and BIC metrics, the values from Model #3 suggest a significant decrease in AIC and a smaller decrease in BIC compared to both Model #1 and Model #2. The smaller decrease in BIC could be expected as penalty for using a larger number of features.

From these results, it seems that the inclusion of additional features in the model, helped enhancing the explanatory power of the model reflected from an increase in the adjusted R-squared value and decreased AIC and BIC values